In [ ]:
-- check the total rows in data table
SELECT count(*)
from vision_zero_crash
WHERE "YEAR" >= 2011

(1 row(s) affected)

Total execution time: 00:00:09.027

count
64188


# Vision Zero Project - San Jose, CA

## SQL - Azure Data Studio

### PostgresSQL queried datasets to be used in Tableau visualization

- (Panel) Crash details for map
- (Panel) Corridor Specific safety details
- (Panel) Top 60 Safety Corridors \w moving average and rolling total


In [ ]:
-- (Panel) Crash details for map

create or replace function violation(a text) RETURNS text AS $$
BEGIN
    IF  position(',' in a) > 0 then
        DECLARE
        v text;
        arr text[] = string_to_array(a,',');
        BEGIN
            FOREACH v IN ARRAY arr 
            LOOP
                IF v != 'Unknown' then 
                    return trim(leading ' ' from v);
                END IF;
            END LOOP;
        END;
    ELSE
      return a;
    END IF;
END; 
$$ language plpgsql;


SELECT 
to_date("CRASHDATETIME",'YYYY/MM/DD') as "CRASHDATE",
violation("VEHICLEVIOLATIONCODEDESCRIPTION") as "VIOLATION",
(CASE
WHEN SPLIT_PART("AGERANGE", ',', 1) != 'NULL' then SPLIT_PART("AGERANGE", ',', 1) 
else '30-49' end) as "AGE",
(CASE  
WHEN "LIGHTING" = 'Dusk - Dawn' then 'Dusk/Dawn'
WHEN "LIGHTING" LIKE 'Dark%' then 'Dark'
else 'Daylight' end) as "LIGHTING",
(CASE  
WHEN "VEHICLEPARTYCATEGORY" LIKE '%Pedestrian%' then 'Pedestrian'
WHEN "VEHICLEPARTYCATEGORY" LIKE '%Bicycle%' then 'Bicycle'
else 'Vehicle' end) as "INVOLVING",
"WEATHER",
"DAYOFWEEKNAME",  
"NARRATIVE", 
"VEHICLEINVOLVEDWITH", 
"COLLISIONTYPE",
"MINORINJURIES", 
"MODERATEINJURIES", 
"SEVEREINJURIES",
"FATALINJURIES", 
"ROADWAYCONDITION", 
"ROADWAYSURFACE",
"INTASTREETNAME", 
"INTBSTREETNAME", 
"INTERSECTIONNUMBERINT", 
"PROXIMITYTOINTERSECTION",
"LATITUDE", "LONGITUDE"
FROM vision_zero_crash 
WHERE "YEAR" >= 2011
ORDER by "CRASHDATETIME"::date

Commands completed successfully

(64188 row(s) affected)

Displaying Top 10 rows.

Total execution time: 00:00:11.086

CRASHDATE,VIOLATION,AGE,LIGHTING,INVOLVING,WEATHER,DAYOFWEEKNAME,NARRATIVE,VEHICLEINVOLVEDWITH,COLLISIONTYPE,MINORINJURIES,MODERATEINJURIES,SEVEREINJURIES,FATALINJURIES,ROADWAYCONDITION,ROADWAYSURFACE,INTASTREETNAME,INTBSTREETNAME,INTERSECTIONNUMBERINT,PROXIMITYTOINTERSECTION,LATITUDE,LONGITUDE
2011-01-01,Unsafe U-Turn,16-29,Dark,Vehicle,Clear,Saturday,29 year-old female driver vs unspecified driver,Other Vehicle,Rear End,0,0,0,0,No Unusual Conditions,Dry,ALMA AV,PALM ST,4168,Non-Related,37.3115987,-121.8808112
2011-01-01,Speeding,30-49,Dark,Vehicle,Rain,Saturday,Unspecified driver vs parked car,Parked Vehicle,Other,0,0,0,0,No Unusual Conditions,Wet,MANZANITA DR,SARATOGA AV,87460,Non-Related,37.31164303,-121.97861969
2011-01-01,Speeding,16-29,Daylight,Vehicle,Cloudy,Saturday,21 year-old female driver vs object,Fixed Object,Other,0,1,0,0,No Unusual Conditions,Wet,BLOSSOM HILL RD/DW,COTTLE RD,42668,Non-Related,37.25377914,-121.8031774
2011-01-01,Speeding,30-49,Daylight,Vehicle,Rain,Saturday,Unspecified driver vs object,Fixed Object,Other,0,0,0,0,No Unusual Conditions,Wet,DELMAS AV/SHEPHERD AV,WILLOW ST,50214,Non-Related,37.31217342,-121.8938377
2011-01-01,Speeding,30-49,Dusk/Dawn,Vehicle,Cloudy,Saturday,40 year-old male drunk truck driver vs 29 year-old male driver,Other Vehicle,Broadside,0,0,0,0,No Unusual Conditions,Wet,BAYSHORE FR,JULIAN ST,14960,Non-Related,37.35317711,-121.8672439
2011-01-01,Unsafe Turn Movement,16-29,Daylight,Vehicle,Rain,Saturday,28 year-old male drunk driver vs object,Fixed Object,Other,0,1,0,0,No Unusual Conditions,Wet,CAPITOL EX,QUIMBY RD,32922,Non-Related,37.3238552,-121.8084406
2011-01-01,Unsafe Turn Movement,30-49,Dark,Vehicle,Cloudy,Saturday,Unspecified driver vs 2 parked cars,Parked Vehicle,Other,0,0,0,0,No Unusual Conditions,Dry,ESTEBAN WY,SOLANO DR,57530,Non-Related,37.23211429,-121.7981208
2011-01-01,Unknown,65 plus,Daylight,Vehicle,Clear,Saturday,84 year-old male driver vs unspecified driver,Other Vehicle,Rear End,0,0,0,0,No Unusual Conditions,Dry,KIELY BL,SARATOGA AV,79188,Non-Related,37.32015115,-121.9724316
2011-01-01,Unknown,16-29,Dark,Pedestrian,Rain,Saturday,44 year-old male driver vs 21 year-old male pedestrian with serious injury,Pedestrian,Other,0,0,1,0,No Unusual Conditions,Wet,ELEVENTH ST,SAN CARLOS ST,55813,Non-Related,37.33649044,-121.8764777
2011-01-01,Speeding,30-49,Dusk/Dawn,Vehicle,Rain,Saturday,Unspecified tipsy driver vs 2 parked cars,Parked Vehicle,Other,0,0,0,0,No Unusual Conditions,Wet,MENKER AV,PARKMOOR AV,90474,Non-Related,37.31720424,-121.9216311


In [ ]:
-- (Panel) Corridor Specific safety details

ALTER TABLE safety_corridors 
ADD COLUMN IF NOT EXISTS "STREETNAME" VARCHAR;

UPDATE safety_corridors
SET "STREETNAME" = UPPER("SAFETYPRIO");

UPDATE safety_corridors 
SET "STREETNAME" = REPLACE("STREETNAME", 'AVE', 'AV');

UPDATE safety_corridors 
SET "STREETNAME" = REPLACE("STREETNAME", 'EXPRESSWAY', 'EX');

UPDATE safety_corridors 
SET "STREETNAME" = REPLACE("STREETNAME", 'ROAD', 'RD');

UPDATE safety_corridors 
SET "STREETNAME" = REPLACE("STREETNAME", 'STREET', 'ST');
    
UPDATE safety_corridors 
SET "STREETNAME" = REPLACE("STREETNAME", 'LANE', 'LN');

SELECT *
FROM safety_corridors
ORDER BY 1;


NOTICE: column "STREETNAME" of relation "safety_corridors" already exists, skipping

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

...

Commands completed successfully

(30 row(s) affected)

Displaying Top 10 rows.

Total execution time: 00:00:01.733

id geom OBJECTID FACILITYID INTID PROJECTLIM SAFETYPRIO GRANTFUNDI FUNDINGAGE GRANTAMOUN VOLUME JURISDICTI LASTUPDATE NOTES SHAPE_Leng PRIORITY STREETNAME 1 0105000020B3080000010000000102000000C20100005D250BB89E855741151A19EB94CC3D410F307B40A68557416471419FA4CC3D41C9C19041AA8557414413C81FADCC3D41C9514D5CAE855741364B063AB6CC3D416BECDC11B785574114A5483BC9CC3D41F0C38B2DB78557414521DB77C9CC3D41649B3A49B7855741739D6DB4C9CC3D41E772E964B7855741BD1900F1C9CC3D41EB499880B7855741D1107D2DCACC3D41F620479CB7855741E407FA69CACC3D41CF58FBB7B7855741247861A6CACC3D419E90AFD3B7855741FCE7C8E2CACC3D4189C863EFB78557413058301FCBCC3D41FBFF170BB8855741EE42825BCBCC3D419D98D126B8855741DD2BD497CBCC3D4195CF8542B8855741929110D4CBCC3D41C2673F5EB885574165F54C10CCCC3D415061FE79B88557418157894CCCCC3D411CF9B795B88557412A36B088CCCC3D41BE9071B1B8855741AF14D7C4CCCC3D41E78930CDB885574175F1FD00CDCC3D418382EFE8B88557410F490F3DCDCC3D41A57AAE04B98557416D1B0B79CDCC3D417DD47220B985574182711CB5CDCC3D41702D373CB9855741E9BC02F1CDCC3D418B25F657B98557413D8FFE2CCECC3D41CFDFBF73B9855741AFD8E468CECC3D41B938848FB98557413024CBA4CECC3D41369148ABB985574188EA9BE0CECC3D41FF4A12C7B9855741DEAE6C1CCFCC3D414A04DCE2B9855741DFED2758CFCC3D411DBEA5FEB98557415CB2F893CFCC3D41E6766F1ABA855741306C9ECFCFCC3D4187913E36BA855741D5A9590BD0CC3D4127AB0D52BA85574190DCE946D0CC3D4153C5DC6DBA855741C9948F82D0CC3D41F8DEAB89BA855741F8C71FBED0CC3D41AEF87AA5BA855741CBFAAFF9D0CC3D4122734FC1BA855741BBA62A35D1CC3D41ABED23DDBA855741BF52A570D1CC3D412568F8F8BA855741A2FE1FACD1CC3D412FE2CC14BB855741762585E7D1CC3D4185BDA630BB855741644AEA22D2CC3D411A377B4CBB855741D5EB395ED2CC3D41EE115568BB855741808B8999D2CC3D41D4EC2E84BB855741712BD9D4D2CC3D4184280EA0BB855741FD431310D3CC3D416E02E8BBBB8557416FD9374BD3CC3D41253EC7D7BB8557413AF27186D3CC3D415C79A6F3BB855741DF8596C1D3CC3D4118B4850FBC855741EC93A5FCD3CC3D4151EF642BBC8557417027CA37D4CC3D41EA8A4947BC855741DEAEC372D4CC3D41F4262E63BC85574166BBD2ADD4CC3D4188C2127FBC855741A942CCE8D4CC3D41B25DF79ABC855741AA44B023D5CC3D418D5AE1B6BC8557414DCAA95ED5CC3D413AF5C5D2BC85574160477899D5CC3D41A2F1AFEEBC85574192475CD4D5CC3D4194ED990ABD855741AFC22A0FD6CC3D410DE98326BD85574179B8E349D6CC3D4153467342BD855741CB31B284D6CC3D419AA2625EBD85574189A055BFD6CC3D41149E4C7ABD85574199960EFAD6CC3D41B75B4196BD8557418B03B234D7CC3D4176B730B2BD85574127ED3F6FD7CC3D41DA1320CEBD855741165CE3A9D7CC3D4176D014EABD855741E1BE5BE4D7CC3D41A38D0906BE855741B9A6E91ED8CC3D414C4AFE21BE8557417A096259D8CC3D41CE67F83DBE85574110E5C493D8CC3D418124ED59BE855741CD473DCED8CC3D410442E775BE8557415B23A008D9CC3D41045FE191BE855741C579ED42D9CC3D41147CDBADBE85574136D03A7DD9CC3D410799D5C9BE855741A42688B7D9CC3D41E416D5E5BE85574130F6BFF1D9CC3D41CE94D401BF855741A2C5F72BDACC3D413312D41DBF855741D80F1A66DACC3D419E8FD339BF8557412D5A3CA0DACC3D41466ED855BF85574185A25EDADACC3D413BEBD771BF85574172676B14DBCC3D416CC9DC8DBF855741B92A784EDBCC3D4120A7E1A9BF855741B8686F88DBCC3D41F584E6C5BF855741C9A666C2DBCC3D41FCC3F0E1BF8557410EE35DFCDBCC3D414AA1F5FDBF855741199C3F36DCCC3D41CDDFFF19C0855741D4522170DCCC3D41651E0A36C0855741100A03AADCCC3D41D2BD1952C0855741DD39CFE3DCCC3D4181FB236EC085574184E6851DDDCC3D41819A338AC085574140913C57DDCC3D415D3943A6C0855741123CF390DDCC3D414AD852C2C0855741EFE6A9CADDCC3D41D17662DEC0855741790C4B04DECC3D411F7677FAC08557413AABD63DDECC3D41E6758C16C1855741C2CE7777DECC3D41EB139C32C18557415A6F03B1DECC3D411374B64EC1855741C98679EADECC3D41DA72CB6AC185574125A0EF23DFCC3D4103D3E586C1855741CCB7655DDFCC3D415DD1FAA2C1855741164CC696DFCC3D41013115BFC185574149DE26D0DFCC3D4140902FDBC18557417EEB7109E0CC3D41A4D4275AD5855741C8DF9FD90CCD3D4189264F5A0486574136660A9579CD3D419E6A7E8D1B8657416116AF32AFCD3D41DF2FD51B21865741615FAF33BCCD3D4173C817B128865741F48CB001CDCD3D41C4331A0F50865741F1A7C0EE26CE3D41F87301B57C86574176EDD3DC8DCE3D412C4D4AD19D865741DDA79915D9CE3D41B0E338ACA2865741A8553A63E4CE3D41C56FA353A6865741E480B530EDCE3D412A1827AEA98657415FE2DFCAF4CE3D4114041A3FE7865741C7E01D6F82CF3D41C9CD3140EB865741EDA51D568BCF3D41B66660DEFB8657413B55D4A5B1CF3D418BA551900E8757411BE407A

In [ ]:
-- (Panel) Top 60 Safety Corridors \w moving average and rolling total

BEGIN;

CREATE TEMPORARY TABLE Astreet ON COMMIT DROP
as SELECT 
SUM("MINORINJURIES") as minorA, 
SUM("MODERATEINJURIES") as moderateA, 
SUM("SEVEREINJURIES") as severeA,
SUM("FATALINJURIES") as fatalA, 
"INTASTREETNAME",
"YEAR", "MONTHNAME"
FROM vision_zero_crash 
WHERE "YEAR" >= 2011
GROUP BY "INTASTREETNAME", "YEAR", "MONTHNAME";


CREATE TEMPORARY TABLE Bstreet ON COMMIT DROP
as SELECT 
SUM("MINORINJURIES") as minorB, 
SUM("MODERATEINJURIES") as moderateB, 
SUM("SEVEREINJURIES") as severeB,
SUM("FATALINJURIES") as fatalB, 
"INTBSTREETNAME",
"YEAR", "MONTHNAME"
FROM vision_zero_crash 
WHERE "YEAR" >= 2011
GROUP BY "INTBSTREETNAME", "YEAR", "MONTHNAME";


CREATE TEMPORARY TABLE crashes ON COMMIT DROP
as SELECT 
(coalesce(astreet."YEAR", bstreet."YEAR")) as "YEAR",
(coalesce(astreet."MONTHNAME", bstreet."MONTHNAME")) as "MONTHNAME",
(coalesce(astreet.minorA, 0) + coalesce(bstreet.minorB, 0)) as "MINORINJURIES",
(coalesce(astreet.moderateA, 0) + coalesce(bstreet.moderateB, 0)) as "MODERATEINJURIES",
(coalesce(astreet.severeA, 0) + coalesce(bstreet.severeB, 0)) as "SEVEREINJURIES",
(coalesce(astreet.fatalA, 0) + coalesce(bstreet.fatalB, 0)) as "FATALINJURIES",
(coalesce(astreet.minorA, 0) + coalesce(bstreet.minorB, 0) + coalesce(astreet.moderateA, 0) + coalesce(bstreet.moderateB, 0)) as "MMI",
(coalesce(astreet.fatalA, 0) + coalesce(bstreet.fatalB, 0) + coalesce(astreet.severeA, 0) + coalesce(bstreet.severeB, 0)) as "KSI",
CASE WHEN bstreet."INTBSTREETNAME" IS NULL THEN astreet."INTASTREETNAME" ELSE bstreet."INTBSTREETNAME" END AS "STREETNAME"
FROM Astreet astreet
full outer join Bstreet bstreet
ON bstreet."INTBSTREETNAME" = astreet."INTASTREETNAME"
and bstreet."YEAR" = astreet."YEAR"
and bstreet."MONTHNAME" = astreet."MONTHNAME"
ORDER BY "STREETNAME", "YEAR", 
CASE astreet."MONTHNAME" 
WHEN 'January' then 1
WHEN 'February' then 2
WHEN 'March' then 3
WHEN 'April' then 4
WHEN 'May' then 5
WHEN 'June' then 6
WHEN 'July' then 7
WHEN 'August' then 8
WHEN 'September' then 9
WHEN 'October' then 10
WHEN 'November' then 11
else 12 end;


CREATE TEMPORARY TABLE topstreets ON COMMIT DROP
as SELECT 
(CASE  
WHEN "STREETNAME" in (
'MONTEREY RD', 
'ALUM ROCK AV',
'CAPITOL EX',
'BLOSSOM HILL RD',
'JACKSON AV',
'MCKEE RD',
'SENTER RD',
'TULLY RD',
'WHITE RD',
'ALMADEN EX',
'MCLAUGHLIN AV',
'BRANHAM LN',
'KING RD',
'STORY RD',
'SANTA CLARA ST',
'SARATOGA AV',
'FRUITDALE AV',
'HILLSDALE AV',
'FIRST ST') then 'True' 
else 'False' end) as "PRIORITY",
SUM("KSI") as "KSI", "STREETNAME"
FROM crashes
GROUP BY "STREETNAME"
ORDER BY "KSI" DESC
LIMIT 60;

CREATE TEMPORARY TABLE safetycorridors ON COMMIT DROP
as SELECT *
from topstreets
where "KSI" > 30 or "PRIORITY" = 'True';


SELECT *,
to_date(("YEAR"::text||"MONTHNAME"),'YYYYMonth') as "AGGDATE",
AVG("MINORINJURIES") OVER(PARTITION BY "STREETNAME" ORDER BY "STREETNAME", "YEAR", "MONTHNAME" ROWS BETWEEN 5 PRECEDING AND CURRENT ROW) AS "6M_MOV_MIN",
AVG("MODERATEINJURIES") OVER(PARTITION BY "STREETNAME" ORDER BY "STREETNAME", "YEAR", "MONTHNAME" ROWS BETWEEN 5 PRECEDING AND CURRENT ROW) AS "6M_MOV_MOD",
AVG("MMI") OVER(PARTITION BY "STREETNAME" ORDER BY "STREETNAME", "YEAR", "MONTHNAME" ROWS BETWEEN 5 PRECEDING AND CURRENT ROW) AS "6M_MOV_MMI",
AVG("KSI") OVER(PARTITION BY "STREETNAME" ORDER BY "STREETNAME", "YEAR", "MONTHNAME" ROWS BETWEEN 5 PRECEDING AND CURRENT ROW) AS "6M_MOV_KSI",
SUM("MINORINJURIES") OVER(PARTITION BY "STREETNAME" ORDER BY "STREETNAME", "YEAR", "MONTHNAME") AS "ROLL_TOT_MIN",
SUM("MODERATEINJURIES") OVER(PARTITION BY "STREETNAME" ORDER BY "STREETNAME", "YEAR", "MONTHNAME") AS "ROLL_TOT_MOD",
SUM("MMI") OVER(PARTITION BY "STREETNAME" ORDER BY "STREETNAME", "YEAR", "MONTHNAME") AS "ROLL_TOT_MMI",
SUM("KSI") OVER(PARTITION BY "STREETNAME" ORDER BY "STREETNAME", "YEAR", "MONTHNAME") AS "ROLL_TOT_KSI"
FROM crashes
WHERE "STREETNAME" IN (select "STREETNAME" from safetycorridors);

COMMIT;

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

...

(4204 row(s) affected)

Displaying Top 10 rows.

Commands completed successfully

Total execution time: 00:00:01.780

YEAR,MONTHNAME,MINORINJURIES,MODERATEINJURIES,SEVEREINJURIES,FATALINJURIES,MMI,KSI,STREETNAME,AGGDATE,6M_MOV_MINI,6M_MOV_MODI,6M_MOV_MMI,6M_MOV_KSI,ROLL_TOT_MINI,ROLL_TOT_MODI,ROLL_TOT_MMI,ROLL_TOT_KSI
2011,April,15,1,0,0,16,0,ALMADEN EX,2011-04-01,15.0000000000000000,1.00000000000000000000,16.0000000000000000,0E-20,15,1,16,0
2011,August,11,0,0,0,11,0,ALMADEN EX,2011-08-01,13.0000000000000000,0.50000000000000000000,13.5000000000000000,0E-20,26,1,27,0
2011,December,13,6,0,0,19,0,ALMADEN EX,2011-12-01,13.0000000000000000,2.3333333333333333,15.3333333333333333,0E-20,39,7,46,0
2011,February,2,1,0,0,3,0,ALMADEN EX,2011-02-01,10.2500000000000000,2.0000000000000000,12.2500000000000000,0E-20,41,8,49,0
2011,January,8,0,0,0,8,0,ALMADEN EX,2011-01-01,9.8000000000000000,1.6000000000000000,11.4000000000000000,0E-20,49,8,57,0
2011,July,5,1,0,0,6,0,ALMADEN EX,2011-07-01,9.0000000000000000,1.5000000000000000,10.5000000000000000,0E-20,54,9,63,0
2011,June,5,0,0,0,5,0,ALMADEN EX,2011-06-01,7.3333333333333333,1.3333333333333333,8.6666666666666667,0E-20,59,9,68,0
2011,March,3,0,0,0,3,0,ALMADEN EX,2011-03-01,6.0000000000000000,1.3333333333333333,7.3333333333333333,0E-20,62,9,71,0
2011,May,1,0,0,0,1,0,ALMADEN EX,2011-05-01,4.0000000000000000,0.33333333333333333333,4.3333333333333333,0E-20,63,9,72,0
2011,November,5,1,0,0,6,0,ALMADEN EX,2011-11-01,4.5000000000000000,0.33333333333333333333,4.8333333333333333,0E-20,68,10,78,0
